In [ ]:
pip install python-resize-image

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread


from keras.models import Sequential,Model
from keras.layers import Dense, Conv2D, Input, MaxPool2D, UpSampling2D, Concatenate, Conv2DTranspose, Add, AvgPool2D
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adam, RMSprop

import tensorflow as tf 

from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
from PIL import Image

from keras.preprocessing.image import array_to_img , img_to_array , load_img ,ImageDataGenerator 

from subprocess import check_output

In [ ]:
data_dir = "../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/png_images/"
mask_dir = "../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/png_masks/"
all_images = os.listdir(data_dir)

In [ ]:
train_images, validation_images = train_test_split(all_images, train_size=0.8, test_size=0.2)

In [ ]:
'''batch_size = 1
img_size = 512
spe_train = len(train_images)/batch_size
spe_validation = len(validation_images)/batch_size'''

In [ ]:
def data_gen_small(data_dir, mask_dir, images, batch_size, dims):
        """
        data_dir: where the actual images are kept
        mask_dir: where the actual masks are kept
        images: the filenames of the images we want to generate batches from
        batch_size: self explanatory
        dims: the dimensions in which we want to rescale our images
        """
        while True:
            batch = np.random.choice(np.arange(len(images)), batch_size)
            imgs = []
            labels = []
            for i in batch:
                # images
                original_img = load_img(data_dir + images[i])
                #plt.imshow(original_img)
                resized_img = resizeimage.resize_contain(original_img, dims+[3])
                #plt.imshow(resized_img)
                array_img = img_to_array(resized_img)/255
                imgs.append(array_img)
                
                # masks
                original_mask = load_img(mask_dir + images[i].split(".")[0] + '.png')
                resized_mask = resizeimage.resize_contain(original_mask, dims+[3])
                array_mask = img_to_array(resized_mask)/255
                labels.append(array_mask[:, :, 0])
            imgs = np.array(imgs)
            labels = np.array(labels)
            #print labels
            yield imgs, labels.reshape(-1, dims[0], dims[1], 1)

In [ ]:
train_gen = data_gen_small(data_dir, mask_dir, train_images, 5, [128,128])
validation_gen = data_gen_small(data_dir, mask_dir, validation_images, 5, [128, 128])

In [ ]:
img,msk = next(train_gen)
plt.imshow(img[0])
print(img[0].shape)


In [ ]:
'''def down(input_layers,filters,pool=True):
    conv1=Conv2D(filters,(2,2),padding="same",activation='relu')(input_layers)
    residual = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    extra=Conv2D(filters,(1,1),padding="same",activation='relu')(input_layers)
    if pool:
        ext = Add()([residual,extra])
        max_pool = MaxPool2D()(ext)
        return max_pool, residual
    else:
        return residual

def up(input_layer, residual, filters):
    filters=int(filters)
    # upsample = UpSampling2D()(input_layer)
    upsample =Conv2DTranspose(filters,(4,4),padding='same',activation='relu',strides=2)(input_layer)
    upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")(upsample)
    concat = Concatenate(axis=3)([residual, upconv])
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(concat)
    conv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    extra=Conv2D(filters,(1,1),padding="same",activation='relu')(upsample)
    ext = Add()([conv2,extra])
    return ext'''
def down(input_layers,filters,pool=True):
    conv1=Conv2D(filters,(2,2),padding="same",activation='relu')(input_layers)
    residual = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    if pool:
        max_pool = AvgPool2D()(residual)
        return max_pool, residual
    else:
        return residual

def up(input_layer, residual, filters):
    filters=int(filters)
    upsample = UpSampling2D()(input_layer)
    upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")(upsample)
    concat = Concatenate(axis=3)([residual, upconv])
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(concat)
    conv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    return conv2


In [ ]:
'''filters = 16
input_layer = Input(shape = [512, 512, 3])
layers = [input_layer]
residuals = []


# Adding few extra layers
d00, res00 = down(input_layer, filters)
residuals.append(res00)

filters *= 2

# next
d0, res0 = down(d00, filters)
# d2, res2_cur = down(d1, filters)
# res2_pre = Conv2D(filters,(2,2),padding="same",activation='relu')(res1)
res0 = Concatenate(axis=3)([d00, res0])
residuals.append(res0)

filters *=2


# Down 1, 128
d1, res1 = down(d0, filters)
# res1 = Concatenate(axis=3)([d0, res1])
residuals.append(res1)

filters *= 2

# Down 2, 64
d2, res2 = down(d1, filters)
# d2, res2_cur = down(d1, filters)
# res2_pre = Conv2D(filters,(2,2),padding="same",activation='relu')(res1)
res2 = Concatenate(axis=3)([d1, res2])
residuals.append(res2)

filters *= 2

# Down 3, 32
d3, res3 = down(d2, filters)
# d2, res2_cur = down(d1, filters)
# res2_pre = Conv2D(filters,(2,2),padding="same",activation='relu')(res1)
res3 = Concatenate(axis=3)([d2, res3])
residuals.append(res3)

filters *= 2

# Down 4, 16
d4, res4 = down(d3, filters)
# d2, res2_cur = down(d1, filters)
# res2_pre = Conv2D(filters,(2,2),padding="same",activation='relu')(res1)
res4 = Concatenate(axis=3)([d3, res4])
residuals.append(res4)

filters *= 2

# Down 5, 8
d5 = down(d4, filters, pool=False)

# Up 1, 16
up1 = up(d5, residual=residuals[-1], filters=filters/2)

filters /= 2

# Up 2,  32
up2 = up(up1, residual=residuals[-2], filters=filters/2)

filters /= 2

# Up 3, 64
up3 = up(up2, residual=residuals[-3], filters=filters/2)

filters /= 2

# Up 4, 128
up4 = up(up3, residual=residuals[-4], filters=filters/2)


filters /= 2

# Up 5, 256
up5 = up(up4, residual=residuals[-5], filters=filters/2)

filters /= 2

# Up 6, 512
up6 = up(up5, residual=residuals[-6], filters=filters/2)

out = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(up6)

model = Model(input_layer, out)'''
filters = 64
input_layer = Input(shape = [128, 128, 3])
layers = [input_layer]
residuals = []

# Down 1, 128
d1, res1 = down(input_layer, filters)
residuals.append(res1)

filters *= 2

# Down 2, 64
d2, res2 = down(d1, filters)
residuals.append(res2)

filters *= 2

# Down 3, 32
d3, res3 = down(d2, filters)
residuals.append(res3)

filters *= 2

# Down 4, 16
d4, res4 = down(d3, filters)
residuals.append(res4)

filters *= 2

# Down 5, 8
d5 = down(d4, filters, pool=False)

# Up 1, 16
up1 = up(d5, residual=residuals[-1], filters=filters/2)

filters /= 2

# Up 2,  32
up2 = up(up1, residual=residuals[-2], filters=filters/2)

filters /= 2

# Up 3, 64
up3 = up(up2, residual=residuals[-3], filters=filters/2)

filters /= 2

# Up 4, 128
up4 = up(up3, residual=residuals[-4], filters=filters/2)

out = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(up4)

model = Model(input_layer, out)


In [ ]:
model.summary()

In [ ]:
def dice_coef(y_true, y_pred):
    smooth = 1e-5
    
    y_true = tf.round(tf.reshape(y_true, [-1]))
    y_pred = tf.round(tf.reshape(y_pred, [-1]))
    
    isct = tf.reduce_sum(y_true * y_pred)
    
    return 2 * isct / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred))

In [ ]:
model.compile(optimizer=RMSprop(1e-4), loss='binary_crossentropy', metrics=[dice_coef])

In [ ]:
model.fit_generator(train_gen, verbose=1, steps_per_epoch=100, epochs=10)

In [ ]:
model.evaluate_generator(validation_gen,100)

In [ ]:
import keras
import tensorflow as tf
from keras.models import Model
from keras import backend as K
from keras.layers import Input, merge, Conv2D, ZeroPadding2D, UpSampling2D, Dense, concatenate, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import BatchNormalization, Dropout, Flatten, Lambda
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.optimizers import Adam, RMSprop, SGD
from keras.regularizers import l2
from keras.layers.noise import GaussianDropout

import numpy as np

In [ ]:
deep_supervision = True
img_input = Input(shape=(128,128,1))

#Encoder Units
conv1_1 = Conv2D(16, (3, 3), activation='relu', padding='same')(img_input)
conv1_1 = Dropout(0.2)(conv1_1)
conv1_1 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1_1)
pool1_1 = MaxPooling2D((2, 2))(conv1_1)

conv2_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1_1)
conv2_1 = Dropout(0.2)(conv2_1)
conv2_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2_1)
pool2_1 = MaxPooling2D((2, 2))(conv2_1)

#Skip Connection Unit-1
up1_2 = Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')(conv2_1)
up1_2 = concatenate([up1_2, conv1_1])
conv1_2 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1_2)
conv1_2 = Dropout(0.2)(conv1_2)
conv1_2 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1_2)

conv3_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2_1)
conv3_1 = Dropout(0.2)(conv3_1)
conv3_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3_1)
pool3_1 = MaxPooling2D((2, 2))(conv3_1)

#Skip Connection Unit-2 and Unit-3
up2_2 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(conv3_1)
up2_2 = concatenate([up2_2, conv2_1])
conv2_2 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2_2)
conv2_2 = Dropout(0.2)(conv2_2)
conv2_2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2_2)

up1_3 = Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same')(conv2_2)
up1_3 = concatenate([up1_3, conv1_1, conv1_2])
conv1_3 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1_3)
conv1_3 = Dropout(0.2)(conv1_3)
conv1_3 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1_3)

#The bridge between the Encoder and Decoder units
conv4_1 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3_1)
conv4_1 = Dropout(0.2)(conv4_1)
conv4_1 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4_1)

#Decoder Units
up3_2 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same")(conv4_1)
up3_2 = concatenate([up3_2, conv3_1])
conv3_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3_2)
conv3_2 = Dropout(0.2)(conv3_2)
conv3_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3_2)


up2_3 = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding="same")(conv3_2)
up2_3 = concatenate([up2_3, conv2_1, conv2_2])
conv2_3 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2_3)
conv2_3 = Dropout(0.2)(conv2_3)
conv2_3 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2_3)


up1_4 = Conv2DTranspose(16, (3, 3), strides=(2, 2), padding="same")(conv2_3)
up1_4 = concatenate([up1_4, conv1_1, conv1_2, conv1_3])
conv1_4 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1_4)
conv1_4 = Dropout(0.2)(conv1_4)
conv1_4 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1_4)

#Nested Output for deepsupervision
nestnet_output_1 = Conv2D(1, (1, 1), activation='sigmoid', name='output_1', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_2)
nestnet_output_2 = Conv2D(1, (1, 1), activation='sigmoid', name='output_2', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_3)
nestnet_output_3 = Conv2D(1, (1, 1), activation='sigmoid', name='output_3', kernel_initializer = 'he_normal', padding='same', kernel_regularizer=l2(1e-4))(conv1_4)

if deep_supervision:
        model = Model(input=img_input, output=[nestnet_output_1,
                                               nestnet_output_2,
                                               nestnet_output_3])
else:
        model = Model(input=img_input, output=[nestnet_output_3])

In [ ]:
model.summary()